In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
years = ["98", "99", "00"]
#years = ["00"]

In [ ]:
!make

In [ ]:
columns = ["Year", "Exporter", "sitc4", "Value"]
dfs = {
    year: pd.read_sas(f"wtf_data/wtf{year}.sas7bdat")
    for year in years
}
for year, df in dfs.items():
    cond = (
        dfs[year].sitc4.notnull() &
        (dfs[year].Importer == b"World") &
        (dfs[year].Exporter != b"World") &
        (dfs[year][dfs[year].sitc4.notnull()].sitc4.apply(lambda s: not (s.endswith(b"X") or s.endswith(b"A"))))
    )
    dfs[year] = dfs[year][cond]
    dfs[year] = dfs[year][columns].copy()
    dfs[year]["Exporter"] = dfs[year].Exporter.apply(lambda s: s.decode())
    dfs[year]["sitc4"] = dfs[year].sitc4.apply(lambda s: s.decode())


In [ ]:
df = pd.concat(dfs.values())
df

In [ ]:
df[df.sitc4.apply(lambda s: not (s.endswith("X") or s.endswith("A")))].sitc4.nunique()

In [ ]:
exports = df.groupby(["Year", "Exporter", "sitc4"]).sum().reset_index()
exports

In [ ]:
total_country = exports[["Year", "Exporter", "Value"]].groupby(["Year", "Exporter"]).sum().rename(columns={"Value": "TotalCountry"})
total_country

In [ ]:
total_product = exports[["Year", "sitc4", "Value"]].groupby(["Year", "sitc4"]).sum().rename(columns={"Value": "TotalProduct"})
total_product

In [ ]:
total = exports.groupby("Year")["Value"].sum().rename("Total")
total

In [ ]:
exports_totals = exports.join(total_country, on=["Year", "Exporter"]).join(total_product, on=["Year", "sitc4"]).join(total, on="Year")
exports_totals["RCA"] = (exports_totals.Value / exports_totals.TotalCountry) / (exports_totals.TotalProduct / exports_totals.Total)
exports_totals

In [ ]:
pivots = exports_totals.pivot(index="Year", columns=["sitc4", "Exporter"], values="RCA")
matrices = {}
for year in pivots.index:
    X = pivots.loc[year].to_frame().unstack().to_numpy()
    X = 1 * (X > 1)
    matrices[year] = X

In [ ]:
def exports_to_phi(X):
    phi = X @ X.T
    # cantidad de paises que exportan cada producto
    S = phi.diagonal()
    # matriz que en cada fila tiene a S
    M = np.reshape(np.ones_like(S), (-1, 1)) @ np.reshape(S, (1, -1))
    # M[i,j] = max(sumatoria_c Xci, sumatoria_c Xcj)
    # dividir por el maximo me va a dar el minimo
    M = (M * (M >= M.T)) + (M.T * (M < M.T))
    # cuenta final,out y where hacen que la probabilidad sea 0 cuando se intenta dividir por 0
    # esto pasa cuando ninguno de los dos productos se exportan en ningun pais
    return np.divide(phi, M, out=np.zeros_like(phi, dtype=np.float64), where=(M != 0))

In [ ]:
phis = {
    year: exports_to_phi(X)
    for year, X in matrices.items()
}
phis[2000]

In [ ]:
phi = (phis[1998] + phis[1999] + phis[2000]) / 3
phi

In [ ]:
omegas = [x/100 for x in range(30, 81, 5)]

def largest_comp(omega):
    G = nx.from_numpy_matrix(1* (phi > omega))
    return len(max(nx.connected_components(G), key=len)) / len(G)

plt.plot(omegas, list(map(largest_comp, omegas)))
plt.grid()

In [ ]:
def links_by_threshold(t):
    return (1* (phi > t)).sum()
ts = 10.0**(np.arange(-2,0, 0.01))
plt.ylim(10**2.8, 10**6)
plt.xlim(10**-2, 1)
plt.grid()
plt.loglog(ts, list(map(links_by_threshold, ts)))

In [ ]:
plt.xscale('log')
plt.hist(phi.ravel(),bins=10.0**(np.arange(-3,0, 0.1)))

In [ ]:
def iterate(exports, omega):
    return exports | (1 * ((phi @ np.diagflat(exports)).max(1) >= omega))

def simulate(exports, omega):
    simu = [exports]
    for _ in range(4):
        simu.append(iterate(simu[-1], omega))
    return simu

In [ ]:
countries = [c for _, c in pivots.iloc[0].to_frame().unstack().columns.to_list()]
chl_exports = matrices[2000][:, countries.index("Chile")]

In [ ]:
print([s.sum() for s in simulate(chl_exports, 0.55)])
print([s.sum() for s in simulate(chl_exports, 0.60)])
print([s.sum() for s in simulate(chl_exports, 0.65)])

In [ ]:
mst = nx.maximum_spanning_tree(nx.from_numpy_array(phi))
G = nx.compose(mst, nx.from_numpy_array(1* (phi > 0.55)))
G.remove_edges_from(nx.selfloop_edges(G))
#G = mst

In [ ]:
def product_discovery(exports0, omega):
    results = simulate(exports0, omega)
    number_exports = [exports * (i+1) for i, exports in enumerate(results)]
    return pd.DataFrame(number_exports).replace({'0': np.nan, 0: np.nan}).min() - 1

In [ ]:
def plot(product_discovery, title):
    plt.figure(figsize=(12, 10))

    # Fue la mejor forma que encontré de poner un color distinto a los NaN (productos que nunca se llegan a desarrollar)
    color_dict = {
        0: "green",
        1: "greenyellow",
        2: "yellow",
        3: "orange",
        4: "red"
    }

    def select_color(value):
        return color_dict.get(value, "black")
    #kamada_kawai
    nx.draw(G, pos=nx.kamada_kawai_layout(G),
            node_size=40, edge_color='gray',
            node_color=product_discovery.apply(select_color))

    for iterations, color in color_dict.items():
        plt.plot([0], [0], 'o', color=color, label=iterations)
    plt.plot([0], [0], 'o', color='white')
    plt.legend(title="Pasos de simulación")
    plt.title(title)

In [ ]:
plot(product_discovery(chl_exports, 0.55), "Chile, Omega=0.55")

In [ ]:
plot(product_discovery(chl_exports, 0.60), "Chile, Omega=0.60")

In [ ]:
plot(product_discovery(chl_exports, 0.65), "Chile, Omega=0.65")

In [ ]:
kor_exports = matrices[2000][:, countries.index("Korea Rep.")]

In [ ]:
plot(product_discovery(kor_exports, 0.55), "Corea del Sur, Omega=0.55")

In [ ]:
plot(product_discovery(kor_exports, 0.60), "Corea del Sur, Omega=0.60")

In [ ]:
plot(product_discovery(kor_exports, 0.65), "Corea del Sur, Omega=0.65")